In [1]:
import pandas as pd
import requests

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [3]:



API_KEY = "8edf5a3557214b83a8d24412250109"
local = "Curitiba"  # pode ser "Curitiba, Brazil" ou latitude
url = "http://api.weatherapi.com/v1/forecast.json"
params = {
    "key": API_KEY,
    "q": local,
    "lang": "pt",     # opcional: traduz descrições para português
    "aqi": "no"       # opcional: inclui dados de qualidade do ar
}

try:
    response = requests.get(url, params=params)
    response.raise_for_status()
    dados = response.json()
    
    # Exemplo de como acessar alguns dados:
    # temperatura = dados["current"]["temp_c"]
    # descricao = dados["current"]["condition"]["text"]
    # umidade = dados["current"]["humidity"]
    # vento = dados["current"]["wind_kph"]

    # print(f"Tempo em {dados['location']['name']}: {descricao.capitalize()}")
    # print(f"Temperatura: {temperatura} °C")
    # print(f"Umidade: {umidade}%")
    # print(f"Vento: {vento} km/h")
   # print(dados)

except requests.exceptions.HTTPError as err:
    print(f"Erro HTTP: {err} — código: {response.status_code}")
except ValueError:
    print("Erro ao decodificar JSON.")
except Exception as e:
    print(f"Ocorreu um erro: {e}")

df = pd.json_normalize(dados)
df

    


,location.name,location.region,location.country,location.lat,location.lon,location.tz_id,location.localtime_epoch,location.localtime,current.last_updated_epoch,current.last_updated,current.temp_c,current.temp_f,current.is_day,current.condition.text,current.condition.icon,current.condition.code,current.wind_mph,current.wind_kph,current.wind_degree,current.wind_dir,current.pressure_mb,current.pressure_in,current.precip_mm,current.precip_in,current.humidity,current.cloud,current.feelslike_c,current.feelslike_f,current.windchill_c,current.windchill_f,current.heatindex_c,current.heatindex_f,current.dewpoint_c,current.dewpoint_f,current.vis_km,current.vis_miles,current.uv,current.gust_mph,current.gust_kph,forecast.forecastday
0,Curitiba,Parana,Brazil,-25.4167,-49.25,America/Sao_Paulo,1762707564,2025-11-09 13:59,1762706700,2025-11-09 13:45,17.3,63.1,1,Parcialmente nublado,//cdn.weatherapi.com/weather/64x64/day/116.png,1003,4.7,7.6,104,ESE,1022.0,30.18,0.01,0.0,68,75,17.3,63.1,16.6,61.9,16.6,61.9,10.2,50.3,10.0,6.0,8.7,5.6,9.0,"[{'date': '2025-11-09', 'date_epoch': 17626464..."


In [4]:
forecast_list = dados["forecast"]["forecastday"]
df_forecast = pd.json_normalize(forecast_list)
df_forecast


,date,date_epoch,hour,day.maxtemp_c,day.maxtemp_f,day.mintemp_c,day.mintemp_f,day.avgtemp_c,day.avgtemp_f,day.maxwind_mph,day.maxwind_kph,day.totalprecip_mm,day.totalprecip_in,day.totalsnow_cm,day.avgvis_km,day.avgvis_miles,day.avghumidity,day.daily_will_it_rain,day.daily_chance_of_rain,day.daily_will_it_snow,day.daily_chance_of_snow,day.condition.text,day.condition.icon,day.condition.code,day.uv,astro.sunrise,astro.sunset,astro.moonrise,astro.moonset,astro.moon_phase,astro.moon_illumination,astro.is_moon_up,astro.is_sun_up
0,2025-11-09,1762646400,"[{'time_epoch': 1762657200, 'time': '2025-11-0...",16.9,62.4,10.3,50.6,13.3,56.0,7.6,12.2,0.61,0.02,0.0,9.3,5.0,82,1,88,0,0,Possibilidade de chuva irregular,//cdn.weatherapi.com/weather/64x64/day/176.png,1063,1.4,05:24 AM,06:38 PM,11:28 PM,09:01 AM,Waning Gibbous,83,1,0


In [5]:
# Extrair a lista de horas do primeiro dia de previsão
hour_list = df_forecast['hour'][0]  # Pega a lista de horas do primeiro dia

# Agora normalize essa lista para criar o dataframe de horas
df_hour = pd.json_normalize(hour_list)

# Converter para datetime
df_hour["time"] = pd.to_datetime(df_hour["time"])

# Definir como índice
df_hour = df_hour.set_index("time")

# Selecionar apenas colunas numéricas
df_numeric = df_hour.select_dtypes(include="number")

# Reamostrar de 5h em 5h e calcular média
df_5h = df_numeric.resample("5h").mean().reset_index()

print(df_5h.head())
df_5h

                 time    time_epoch  temp_c  temp_f  is_day  wind_mph  \
0 2025-11-09 00:00:00  1.762664e+09  11.760  53.180     0.0      6.32   
1 2025-11-09 05:00:00  1.762682e+09  12.700  54.840     0.8      5.30   
2 2025-11-09 10:00:00  1.762700e+09  16.820  62.260     1.0      5.14   
3 2025-11-09 15:00:00  1.762718e+09  14.160  57.500     0.8      7.20   
4 2025-11-09 20:00:00  1.762735e+09  10.925  51.675     0.0      6.15   

   wind_kph  wind_degree  pressure_mb  pressure_in  precip_mm  precip_in  \
0    10.140       119.40       1016.2      30.0080      0.028        0.0   
1     8.580       119.20       1019.4      30.1060      0.074        0.0   
2     8.280       104.00       1019.8      30.1140      0.014        0.0   
3    11.600       104.40       1019.2      30.1000      0.006        0.0   
4     9.875        95.75       1021.5      30.1625      0.000        0.0   

   snow_cm  humidity  cloud  feelslike_c  feelslike_f  windchill_c  \
0      0.0     88.40   82.6       

,time,time_epoch,temp_c,temp_f,is_day,wind_mph,wind_kph,wind_degree,pressure_mb,pressure_in,precip_mm,precip_in,snow_cm,humidity,cloud,feelslike_c,feelslike_f,windchill_c,windchill_f,heatindex_c,heatindex_f,dewpoint_c,dewpoint_f,will_it_rain,chance_of_rain,will_it_snow,chance_of_snow,vis_km,vis_miles,gust_mph,gust_kph,uv,condition.code
0,2025-11-09 00:00:00,1.762664e+09,11.760,53.180,0.0,6.32,10.140,119.40,1016.2,30.0080,0.028,0.0,0.0,88.40,82.6,10.740,51.30,10.740,51.30,11.76,53.180,9.960,49.900,0.6,53.2,0.0,0.0,10.0,6.0,8.620,13.86,0.00,1040.2
1,2025-11-09 05:00:00,1.762682e+09,12.700,54.840,0.8,5.30,8.580,119.20,1019.4,30.1060,0.074,0.0,0.0,87.00,89.0,12.060,53.74,12.060,53.74,12.70,54.840,10.500,50.920,0.8,92.8,0.0,0.0,10.0,6.0,6.780,10.88,0.50,1063.0
2,2025-11-09 10:00:00,1.762700e+09,16.820,62.260,1.0,5.14,8.280,104.00,1019.8,30.1140,0.014,0.0,0.0,65.00,88.4,16.680,62.02,16.680,62.02,16.68,62.020,9.940,49.900,1.0,77.8,0.0,0.0,10.0,6.0,6.020,9.70,5.26,1051.0
3,2025-11-09 15:00:00,1.762718e+09,14.160,57.500,0.8,7.20,11.600,104.40,1019.2,30.1000,0.006,0.0,0.0,80.60,69.0,13.540,56.38,13.540,56.38,14.16,57.520,10.780,51.380,0.4,46.6,0.0,0.0,10.0,6.0,9.440,15.16,1.12,1039.6
4,2025-11-09 20:00:00,1.762735e+09,10.925,51.675,0.0,6.15,9.875,95.75,1021.5,30.1625,0.000,0.0,0.0,93.25,76.5,9.775,49.60,9.775,49.60,10.95,51.675,9.925,49.875,0.0,0.0,0.0,0.0,6.0,3.5,9.075,14.60,0.00,1018.0


In [6]:
colunas_importantes = [
    "time",
    "feelslike_c",
    "humidity",
    "cloud",
    "uv",
    "wind_kph",
    "will_it_rain",
    "chance_of_rain",
]

df_limpo = df_5h[[c for c in colunas_importantes if c in df_5h.columns]]
df_limpo.sample(5)

,time,feelslike_c,humidity,cloud,uv,wind_kph,will_it_rain,chance_of_rain
3,2025-11-09 15:00:00,13.540,80.60,69.0,1.12,11.600,0.4,46.6
1,2025-11-09 05:00:00,12.060,87.00,89.0,0.50,8.580,0.8,92.8
4,2025-11-09 20:00:00,9.775,93.25,76.5,0.00,9.875,0.0,0.0
0,2025-11-09 00:00:00,10.740,88.40,82.6,0.00,10.140,0.6,53.2
2,2025-11-09 10:00:00,16.680,65.00,88.4,5.26,8.280,1.0,77.8


In [7]:
import os
print(os.getcwd())

c:\Users\diona\OneDrive\Área de Trabalho\wardobre_ipynb\wardobre_ipynb\PROJETO\BACKEND
